In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [4]:
config = load_config()
od = '../../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [44]:
meta_file = '../config.tsv'
meta_file_2 = 'config.tsv'
genomes_file = 'genomes_config.tsv'

df = parse_config(meta_file)
df2 = pd.read_csv(meta_file_2, sep='\t')
df2['tech_rep'] = df2.cell_line_id+'_1'
df2['same_sample'] = df2['pangenome_code']

# TODO test
# df2 = df2.loc[df2.cell_line_id == 'GM24385']

# get the genomes to download
g_df = pd.read_csv(genomes_file, sep='\t')

# maternal haplotypes
g_df['aws_mat_link'] = g_df['hap2_aws_fasta']
g_df = g_df.loc[g_df['aws_mat_link'].notnull()]
assert len(g_df.loc[g_df['aws_mat_link'].str.contains('maternal')].index) == len(g_df.index)

# paternal haplotypes
g_df['aws_pat_link'] = g_df['hap1_aws_fasta']
g_df = g_df.loc[g_df['aws_pat_link'].notnull()]
assert len(g_df.loc[g_df['aws_pat_link'].str.contains('paternal')].index) == len(g_df.index)


genome_cols = ['same_population_sample', 'european_sample',	'afr_sample', 'same_sample']
assemblies = genome_cols
g_df = g_df.loc[(g_df['sample'].isin(df2[genome_cols[0]]))|
                (g_df['sample'].isin(df2[genome_cols[1]]))|
                (g_df['sample'].isin(df2[genome_cols[2]]))|
                (g_df['sample'].isin(df2[genome_cols[3]]))]

# a little more df2 formatting
df2 = df2[['tech_rep']+assemblies].melt(id_vars='tech_rep')
df2 = df2.reset_index()
df2 = df2.rename({'variable':'assembly_status',
                  'value': 'assembly_sample'},
                  axis=1)


# limit just to the samples where we'll do this
df = df.loc[df.tech_rep.isin(df2.tech_rep.tolist())]

# get a key for assembly status, assembly sample, and actual sample
df2['dataset_key'] = df2.assembly_status+'_'+\
                     df2.assembly_sample+'_'+\
                     df2.tech_rep

In [38]:
df2.loc[df2.tech_rep=='GM18906_1']

,index,tech_rep,assembly_status,assembly_sample,dataset_key
1,1,GM18906_1,same_population_sample,NA19240,same_population_sample_NA19240_GM18906_1
7,7,GM18906_1,european_sample,HG002,european_sample_HG002_GM18906_1
13,13,GM18906_1,afr_sample,HG02717,afr_sample_HG02717_GM18906_1


In [33]:
g_df.head()

,sample,hap1_aws_fasta,hap2_aws_fasta,hap1_gcp_fasta,hap2_gcp_fasta,hap1_fasta_sha256,hap2_fasta_sha256,aws_mat_link,aws_pat_link
2,HG00673,s3://human-pangenomics/working/HPRC/HG00673/as...,s3://human-pangenomics/working/HPRC/HG00673/as...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,9650cdb2f090bb94dcb81d0cf9be27e2565de0489dc0d7...,0eef54ceca964b5a65c046529e5e433f0cb1a19d6d8a19...,s3://human-pangenomics/working/HPRC/HG00673/as...,s3://human-pangenomics/working/HPRC/HG00673/as...
13,HG01928,s3://human-pangenomics/working/HPRC/HG01928/as...,s3://human-pangenomics/working/HPRC/HG01928/as...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,f394a6900c6ba13bb2b9e91ac3d2dcc4863fa5d1fc5a6c...,58fc6831a094299d6913b7d24b495790e70195badddec4...,s3://human-pangenomics/working/HPRC/HG01928/as...,s3://human-pangenomics/working/HPRC/HG01928/as...
14,HG01952,s3://human-pangenomics/working/HPRC/HG01952/as...,s3://human-pangenomics/working/HPRC/HG01952/as...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,c6f0b80ab52d7c58c00fb4d59b46bb18569c00b3bf887a...,5b556ba7058119a135522443eac72f1a084772da2c4209...,s3://human-pangenomics/working/HPRC/HG01952/as...,s3://human-pangenomics/working/HPRC/HG01952/as...
23,HG02717,s3://human-pangenomics/working/HPRC/HG02717/as...,s3://human-pangenomics/working/HPRC/HG02717/as...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,6d7754c4c61594d5ea3aecf9cff9e916d17db4ec727634...,90118c064532b848c572fb9050708c8d53ccce687633ce...,s3://human-pangenomics/working/HPRC/HG02717/as...,s3://human-pangenomics/working/HPRC/HG02717/as...
29,HG002,s3://human-pangenomics/working/HPRC_PLUS/HG002...,s3://human-pangenomics/working/HPRC_PLUS/HG002...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,ec46feb51b343f030aae3e7c4781463a36f059a031f4aa...,9e0b7ae6b0727a8828cef61c7b216eabdb2f098fcd2940...,s3://human-pangenomics/working/HPRC_PLUS/HG002...,s3://human-pangenomics/working/HPRC_PLUS/HG002...


In [7]:
expand(config['lr']['map_personal']['max_mapq_tsv_summary'],
               thresh=[0, 10])

['../../data/map_personal/max_mapq_0_bool_summary.tsv',
 '../../data/map_personal/max_mapq_10_bool_summary.tsv']

In [10]:
expand(expand(config['lr']['map_personal']['mapq_tsv'],
                               zip,
                               assembly_status=df2.assembly_status.tolist(),
                               assembly_sample=df2.assembly_sample.tolist(),
                               sample=df2.tech_rep.tolist(),
                               allow_missing=True),
                               thresh=0)

['../../data/map_personal/GM24385_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM18906_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM19240_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG00621_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01928_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01952_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM24385_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM18906_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM19240_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG00621_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01928_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01952_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM24385_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM18906_1/mapq_0_bool.tsv',
 '../../data/map_personal/GM19240_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG00621_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01928_1/mapq_0_bool.tsv',
 '../../data/map_personal/HG01952_1/mapq_0_bool.tsv']

['../../data/map_personal/same_population_sample/HG002_GM24385_1_mapqs.txt',
 '../../data/map_personal/european_sample/HG002_GM24385_1_mapqs.txt',
 '../../data/map_personal/afr_sample/HG02717_GM24385_1_mapqs.txt']

In [25]:
def get_df_val(df, col1, col_dict, uniq_val=True):
    temp = df.copy(deep=True)

    for key, item in col_dict.items():
        temp = temp.loc[temp[key] == item]

    if uniq_val:
        val = temp[col1].unique()
        assert len(val) == 1
        return val[0]
    else:
        return temp[col1].tolist()

In [29]:
df2.loc[df2.tech_rep==sample].assembly_status.tolist()
get_df_val(df2, 'assembly_status', {'tech_rep':sample}, uniq_val=False),
get_df_val(df2, 'assembly_sample', {'tech_rep':sample}, uniq_val=False)
get_df_val(df2, 'tech_rep', {'tech_rep':sample}, uniq_val=False)
# df2.loc[df2.tech_rep==sample].assembly_sample.tolist()
# df2.loc[df2.tech_rep==sample].tech_rep.tolist()

['GM24385_1', 'GM24385_1', 'GM24385_1']

In [30]:
sample = 'GM24385_1'
expand(config['lr']['map_personal']['bam_mapqs'],
               zip,
               assembly_status=get_df_val(df2, 'assembly_status', {'tech_rep':sample}, uniq_val=False),
               assembly_sample=get_df_val(df2, 'assembly_sample', {'tech_rep':sample}, uniq_val=False),
               sample=get_df_val(df2, 'tech_rep', {'tech_rep':sample}, uniq_val=False))

['../../data/map_personal/same_population_sample/HG002_GM24385_1_mapqs.txt',
 '../../data/map_personal/european_sample/HG002_GM24385_1_mapqs.txt',
 '../../data/map_personal/afr_sample/HG02717_GM24385_1_mapqs.txt']